# Scripts para crear los archivos txt con los inserts

Importanción de librerías:

In [8]:
import pandas as pd
import numpy as np
from itertools import chain
from database import Database

db = Database('../db/fifa25.db')

ModuleNotFoundError: No module named 'pandas'

Ruta global y lectura de los datasets

In [ ]:
BASE_PATH = './../docs/'
df_females = pd.read_csv(f'{BASE_PATH}female_players.csv', sep=',')
df_males = pd.read_csv(f'{BASE_PATH}male_players.csv', sep=',')
df_all = pd.read_csv(f'{BASE_PATH}all_players.csv', sep=',')
df_all.columns

: 

### Llenar Jugadores

Vamos a leer los <b>teams</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE players (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, xyz
  rank INTEGER,xyz
  name VARCHAR(50),xyz
  overall INTEGER,xy
  velocity INTEGER,xy
  shooting INTEGER,xy
  passing INTEGER,xy
  dribbling INTEGER,xy
  defending INTEGER,xy
  physicality INTEGER,xy
  position VARCHAR(50),xyz
  weak_foot INTEGER,xyz
  skill_moves INTEGER,xyz
  foot VARCHAR(50),xyz
  height INTEGER,xyz
  weight INTEGER,xyz
  alternative_positions VARCHAR(50),xyz
  age INTEGER,xyz
  nation VARCHAR(50),xyz
  league VARCHAR(50),xyz
  team VARCHAR(50),xyz
  play_styles VARCHAR(50),xy
  url VARCHAR(120),xyz
  diving INTEGER,xy
  handling INTEGER,xy
  kicking INTEGER,xy
  positioning INTEGER,xy
  reflexes INTEGER,xy
  sex VARCHAR(10)xyz
);
```

In [172]:
male_playes_data = df_males[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2', 'Alternative positions', 'League', 'play style']].to_numpy()
female_playes_data = df_females[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2', 'Alternative positions', 'League', 'play style']].to_numpy()

text_players = '-- migrate:up \n\n'
id = 0
# fill males
for player in male_playes_data:
  # player data
  id = id + 1
  name = player[0].replace("'", "''")
  rank = player[1]
  weak_foot = player[2]
  skill_moves = player[3]
  height = player[4].split(' / ')[0][:-2]
  weight = player[5].split(' / ')[0][:-2]
  url = player[6] + player[24]
  foot = player[7]
  sex = 'Hombre'
  alternative_positions = player[25]
  position = player[8]
  nation = player[9].replace("'", "''")
  team = player[10].replace("'", "''")
  age = player[23]
  league = player[26]
  play_styles = player[27]
  # common data
  overall = player[11]
  velocity = player[12] # PAC = velocidad
  shooting = player[13]
  passing = player[14]
  dribbling = player[15]
  defending = player[16]
  physicality = player[17]
  # gk data
  diving = player[18]
  handling = player[19]
  kicking = player[20]
  positioning = player[21]
  reflexes = player[22]
  # position_id

  text_players += f"INSERT INTO players (id, name, rank, weak_foot, skill_moves, height, weight, url, foot, sex, alternative_positions, position, nation, team, age, league, play_styles, overall, velocity, shooting, passing, dribbling, defending, physicality, diving, handling, kicking, positioning, reflexes) VALUES (  {id}, '{name}', {rank}, {weak_foot}, {skill_moves}, {height}, {weight}, '{url}', '{foot}', '{sex}', '{alternative_positions}', '{position}', '{nation}', '{team}', {age}, '{league}', '{play_styles}', {overall}, {velocity}, {shooting}, {passing}, {dribbling}, {defending}, {physicality}. );\n"

  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
# fill females
for player in female_playes_data:
  # player data
  name = player[0].replace("'", "''")
  rank = player[1]
  weak_foot = player[2]
  skill_moves = player[3]
  height = player[4].split(' / ')[0][:-2]
  weight = player[5].split(' / ')[0][:-2]
  url = player[6] + player[24]
  foot_id = 1 if player[7] == 'Right' else 2
  sex_id = 2
  position = player[8]
  position_id = 0
  nation = player[9].replace("'", "''")
  nation_id = 0
  team = player[10].replace("'", "''")
  team_id = 0
  age = player[23]
  # common data
  # fill males
for player in male_playes_data:
  # player data
  id = id + 1
  name = player[0].replace("'", "''")
  rank = player[1]
  weak_foot = player[2]
  skill_moves = player[3]
  height = player[4].split(' / ')[0][:-2]
  weight = player[5].split(' / ')[0][:-2]
  url = player[6] + player[24]
  foot = player[7]
  sex = 'Mujer'
  alternative_positions = player[25]
  position = player[8]
  nation = player[9].replace("'", "''")
  team = player[10].replace("'", "''")
  age = player[23]
  league = player[26]
  play_styles = player[27]
  # common data
  overall = player[11]
  velocity = player[12] # PAC = velocidad
  shooting = player[13]
  passing = player[14]
  dribbling = player[15]
  defending = player[16]
  physicality = player[17]
  # gk data
  diving = player[18]
  handling = player[19]
  kicking = player[20]
  positioning = player[21]
  reflexes = player[22]
  # position_id

  text_players += f"INSERT INTO players (id, name, rank, weak_foot, skill_moves, height, weight, url, foot, sex, alternative_positions, position, nation, team, age, league, play_styles, overall, velocity, shooting, passing, dribbling, defending, physicality, diving, handling, kicking, positioning, reflexes) VALUES (  {id}, '{name}', {rank}, {weak_foot}, {skill_moves}, {height}, {weight}, '{url}', '{foot}', '{sex}', '{alternative_positions}', '{position}', '{nation}', '{team}', {age}, '{league}', '{play_styles}', {overall}, {velocity}, {shooting}, {passing}, {dribbling}, {defending}, {physicality});\n"
  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# write male an female
text_players = text_players + '\n-- migrate:down \n\nDELETE FROM players;'
with open(f'{BASE_PATH}players2_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)

### Llenar Common Details

Vamos a leer los <b>common details</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE common_details (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  overall INTEGER,
  velocity INTEGER,
  shooting INTEGER,
  passing INTEGER,
  dribbling INTEGER,
  defending INTEGER,
  physicality INTEGER,
  player_id	INTEGER NOT NULL,
  FOREIGN KEY (player_id) REFERENCES players (id)
);
```

In [174]:
male_playes_data = df_males[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2']].to_numpy()
female_playes_data = df_females[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2']].to_numpy()

text_players = '-- migrate:up \n\n'
player_id = 1
id = 1
# fill males
for player in male_playes_data:
  # player data
  position = player[8]
  # common data
  overall = player[11]
  velocity = player[12] # PAC = velocidad
  shooting = player[13]
  passing = player[14]
  dribbling = player[15]
  defending = player[16]
  physicality = player[17]
  # position_id
  # insert player common
  text_players += "INSERT INTO common_details (id, overall, velocity, shooting, passing, dribbling, defending, physicality, player_id) VALUES (" + str(id) + ', ' + str(overall) + ', ' + str(velocity) + ', ' + str(shooting) + ', ' + str(passing) + ', ' + str(dribbling) + ', ' + str(defending) + ', ' + str(physicality) + ', ' + str(player_id) + ");\n"
  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
  id = id + 1
# fill females
for player in female_playes_data:
  position = player[8]
  # common data
  overall = player[11]
  velocity = player[12] # PAC = velocidad
  shooting = player[13]
  passing = player[14]
  dribbling = player[15]
  defending = player[16]
  physicality = player[17]
  # position_id
  # insert player common
  text_players += "INSERT INTO common_details (id, overall, velocity, shooting, passing, dribbling, defending, physicality, player_id) VALUES (" + str(id) + ', ' + str(overall) + ', ' + str(velocity) + ', ' + str(shooting) + ', ' + str(passing) + ', ' + str(dribbling) + ', ' + str(defending) + ', ' + str(physicality) + ', ' + str(player_id) + ");\n"
  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
  id = id + 1

  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
# write male an female
text_players = text_players + '\n-- migrate:down \n\nDELETE FROM common_details;'
with open(f'{BASE_PATH}common_details_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)

### Llenar Goalkeepers Details

Vamos a leer los <b>goalkeepers details</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE goalkeeper_details (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  diving INTEGER,
  handling INTEGER,
  kicking INTEGER,
  positioning INTEGER,
  reflexes INTEGER,
  player_id	INTEGER NOT NULL,
  FOREIGN KEY (player_id) REFERENCES players (id)
);
```

In [176]:
male_playes_data = df_males[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2']].to_numpy()
female_playes_data = df_females[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2']].to_numpy()

text_players = '-- migrate:up \n\n'
player_id = 1
id = 1
# fill males
for player in male_playes_data:
  # player data
  position = player[8]
  # common data
  # gk data
  diving = player[18]
  handling = player[19]
  kicking = player[20]
  positioning = player[21]
  reflexes = player[22]
  # position_id
  # insert player common
  if position == 'GK':
    text_players += "INSERT INTO goalkeeper_details (id, diving, handling, kicking, positioning, reflexes, player_id) VALUES (" + str(id) + ', ' + str(diving) + ', ' + str(handling) + ', ' + str(kicking) + ', ' + str(positioning) + ', ' + str(reflexes) + ', ' + str(player_id) + ");\n"
  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
  id = id + 1
# fill females
for player in female_playes_data:
  position = player[8]
  # common data
  # gk data
  diving = player[18]
  handling = player[19]
  kicking = player[20]
  positioning = player[21]
  reflexes = player[22]
  # position_id
  # insert player common
  if position == 'GK':
    text_players += "INSERT INTO goalkeeper_details (id, diving, handling, kicking, positioning, reflexes, player_id) VALUES (" + str(id) + ', ' + str(diving) + ', ' + str(handling) + ', ' + str(kicking) + ', ' + str(positioning) + ', ' + str(reflexes) + ', ' + str(player_id) + ");\n"
  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
  id = id + 1

  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
  player_id = player_id + 1
# write male an female
text_players = text_players + '\n-- migrate:down \n\nDELETE FROM goalkeeper_details;'
with open(f'{BASE_PATH}goalkeeper_details_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)

### Llenar players_play_styles

Vamos a leer los <b>players_play_styles</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE players_play_styles (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  play_style_id	INTEGER NOT NULL,
  player_id	INTEGER NOT NULL,
  FOREIGN KEY (play_style_id) REFERENCES play_styles (id),
  FOREIGN KEY (player_id) REFERENCES players (id)
);
```

In [185]:
male_playes_data = df_males[['play style']].to_numpy()
female_playes_data = df_females[['play style']].to_numpy()

text_players = '-- migrate:up \n\n'
player_id = 1
id = 1
# fill males
for player in male_playes_data:
  if not pd.isna(player[0]):
    styles = player[0].split(', ')
    for s in styles:
      # play_style_id
      query = f"SELECT id FROM play_styles WHERE name = '{s}'"
      rs = db.fetchone(query)
      if rs is not None:
        play_style_id = rs['id']
        text_players += "INSERT INTO players_play_styles (id, play_style_id, player_id) VALUES (" + str(id) + ', ' + str(play_style_id) + ', ' + str(player_id) + ");\n"
        id = id + 1
      else:
        print(f"PlayStylePlayer '{s}' not found in the database.")
  player_id = player_id + 1
# fill females
for player in female_playes_data:
  if not pd.isna(player[0]):
    styles = player[0].split(', ')
    for s in styles:
      # play_style_id
      query = f"SELECT id FROM play_styles WHERE name = '{s}'"
      rs = db.fetchone(query)
      if rs is not None:
        play_style_id = rs['id']
        text_players += "INSERT INTO players_play_styles (id, play_style_id, player_id) VALUES (" + str(id) + ', ' + str(play_style_id) + ', ' + str(player_id) + ");\n"
        id = id + 1
      else:
        print(f"PlayStylePlayer '{s}' not found in the database.")
  player_id = player_id + 1
# write male an female
text_players = text_players + '\n-- migrate:down \n\nDELETE FROM players_play_styles;'
with open(f'{BASE_PATH}players_play_styles_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)

### Llenar players_positions

Vamos a leer los <b>players_positions</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE players_positions (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  position_id	INTEGER NOT NULL,
  player_id	INTEGER NOT NULL,
  FOREIGN KEY (position_id) REFERENCES positions (id),
  FOREIGN KEY (player_id) REFERENCES players (id)
);
```

In [186]:
male_playes_data = df_males[['Alternative positions']].to_numpy()
female_playes_data = df_females[['Alternative positions']].to_numpy()

text_players = '-- migrate:up \n\n'
player_id = 1
id = 1
# fill males
for player in male_playes_data:
  if not pd.isna(player[0]):
    styles = player[0].split(', ')
    for s in styles:
      # play_style_id
      query = f"SELECT id FROM positions WHERE name = '{s}'"
      rs = db.fetchone(query)
      if rs is not None:
        position_id = rs['id']
        text_players += "INSERT INTO players_positions (id, position_id, player_id) VALUES (" + str(id) + ', ' + str(position_id) + ', ' + str(player_id) + ");\n"
        id = id + 1
      else:
        print(f"Position '{s}' not found in the database.")
  player_id = player_id + 1
# fill females
for player in female_playes_data:
  if not pd.isna(player[0]):
    styles = player[0].split(', ')
    for s in styles:
      # position_id
      query = f"SELECT id FROM positions WHERE name = '{s}'"
      rs = db.fetchone(query)
      if rs is not None:
        position_id = rs['id']
        text_players += "INSERT INTO players_positions (id, position_id, player_id) VALUES (" + str(id) + ', ' + str(position_id) + ', ' + str(player_id) + ");\n"
        id = id + 1
      else:
        print(f"Position '{s}' not found in the database.")
  player_id = player_id + 1
# write male an female
text_players = text_players + '\n-- migrate:down \n\nDELETE FROM players_positions;'
with open(f'{BASE_PATH}players_positions_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)